# Калякин Тимофей
# ДЗ 3

## Создание таблиц

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas as pd

In [2]:
# Подключаемся к основной базе данных
conn = pg.connect(dbname='postgres', host='localhost', user='postgres', password='12345', port=5432)
conn.autocommit = True  
cursor = conn.cursor()

new_db_name = 'hw3_database'

# Завершаем все активные подключения к базе перед удалением
cursor.execute(f"""
    SELECT pg_terminate_backend(pg_stat_activity.pid)
    FROM pg_stat_activity
    WHERE pg_stat_activity.datname = '{new_db_name}'
    AND pid <> pg_backend_pid();
""")

# Удаляем базу данных, если она существует
cursor.execute(f'DROP DATABASE IF EXISTS {new_db_name};')

cursor.close()
conn.close()


In [3]:
conn = pg.connect(dbname='postgres', host='localhost', user='postgres', password='12345', port=5432)
conn.autocommit = True  
cursor = conn.cursor()

new_db_name = 'hw3_database'
cursor.execute(f'CREATE DATABASE {new_db_name};')

cursor.close()
conn.close()


In [4]:
customer_df = pd.read_csv('customer.csv', delimiter=';')
# Создание новой строки с customer_id = 5034 и остальными значениями как NaN
new_row = {col: None for col in customer_df.columns}
new_row['customer_id'] = int(5034)  
customer_df = pd.concat([customer_df, pd.DataFrame([new_row])], ignore_index=True)
display(customer_df)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,NaN,Male,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7
3999,4000,Kippy,Oldland,Male,1991-11-05,Software Engineer IV,NaN,Affluent Customer,N,No,613 Erie Lane,3996,NSW,Australia,10


In [5]:
transaction_df = pd.read_csv('transaction.csv', delimiter=';')
transaction_df['list_price'] = transaction_df['list_price'].astype(str).str.replace(',', '.').astype(float)
transaction_df['standard_cost'] = transaction_df['standard_cost'].astype(str).str.replace(',', '.').astype(float)
transaction_df['customer_id'] = transaction_df['customer_id'].astype(int)
display(transaction_df)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,24.06.2017,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19996,19997,41,127,09.11.2017,True,Approved,Solex,Road,medium,medium,416.98,312.74
19997,19998,87,2284,14.04.2017,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19998,19999,6,2764,03.07.2017,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73


In [6]:
# Функция для проверки таблиц
def check_tables(limit=10):
    # Получаем список всех таблиц в схеме public
    cursor.execute("""
        SELECT table_name FROM information_schema.tables 
        WHERE table_schema='public' AND table_type='BASE TABLE'
    """)
    tables = cursor.fetchall()

    print("Список таблиц в базе данных:")
    
    for table in tables:
        table_name = table[0]
        print(f"\n Таблица: {table_name}")

        # Запрашиваем данные из текущей таблицы
        try:
            cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
            rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]  
            
            df = pd.DataFrame(rows, columns=colnames)
            print(df)

        except Exception as e:
            print(f"Ошибка при получении данных из {table_name}: {e}")

In [7]:
conn = pg.connect(dbname='hw3_database', host='localhost', user='postgres', password='12345', port=5432)
cursor = conn.cursor()

In [8]:
check_tables()

Список таблиц в базе данных:


In [9]:
create_tables_query = '''

CREATE TABLE Customer (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(30),
    dob VARCHAR(50),
    job_title VARCHAR(50),
    job_industry_category VARCHAR(50),
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(30),
    owns_car VARCHAR(30),
    address VARCHAR(50),
    postcode VARCHAR(30),
    state VARCHAR(30),
    country VARCHAR(30),
    property_valuation INT
);

CREATE TABLE Transaction (
    transaction_id INT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    transaction_date VARCHAR(30),
    online_order VARCHAR(30),
    order_status VARCHAR(30),
    brand VARCHAR(30),
    product_line VARCHAR(30),
    product_class VARCHAR(30),
    product_size VARCHAR(30),
    list_price FLOAT,
    standard_cost FLOAT,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id) ON DELETE CASCADE
)
'''

In [10]:
# Выполняем SQL-запрос для создания таблиц
cursor.execute(create_tables_query)
conn.commit()

In [11]:
# Загружаем таблицу Customer
for _, row in customer_df.iterrows():
    cursor.execute("""
        INSERT INTO Customer (
            customer_id, first_name, last_name, gender, dob, job_title, 
            job_industry_category, wealth_segment, deceased_indicator, owns_car, 
            address, postcode, state, country, property_valuation
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (customer_id) DO NOTHING;
    """, (
        row['customer_id'], row['first_name'], row['last_name'], row['gender'], row['DOB'],
        row['job_title'], row['job_industry_category'], row['wealth_segment'],
        row['deceased_indicator'], row['owns_car'], row['address'], row['postcode'],
        row['state'], row['country'], row['property_valuation']
    ))

conn.commit()



In [12]:
# Загружаем таблицу Transaction
for _, row in transaction_df.iterrows():
    cursor.execute("""
        INSERT INTO Transaction (
            transaction_id, product_id, customer_id, transaction_date, online_order, 
            order_status, brand, product_line, product_class, product_size, 
            list_price, standard_cost
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (transaction_id) DO NOTHING;
    """, (
        row['transaction_id'], row['product_id'], row['customer_id'], row['transaction_date'],
        row['online_order'], row['order_status'], row['brand'], row['product_line'],
        row['product_class'], row['product_size'], row['list_price'], row['standard_cost']
    ))
conn.commit()

In [13]:
check_tables()

Список таблиц в базе данных:

 Таблица: customer
   customer_id      first_name  last_name  gender         dob  \
0            1         Laraine  Medendorp       F  1953-10-12   
1            2             Eli    Bockman    Male  1980-12-16   
2            3           Arlin     Dearle    Male  1954-01-20   
3            4          Talbot        NaN    Male  1961-10-03   
4            5  Sheila-kathryn     Calton  Female  1977-05-13   
5            6            Curr  Duckhouse    Male  1966-09-16   
6            7            Fina     Merali  Female  1976-02-23   
7            8             Rod      Inder    Male  1962-03-30   
8            9            Mala       Lind  Female  1973-03-10   
9           10        Fiorenze    Birdall  Female  1988-10-11   

                              job_title job_industry_category  \
0                   Executive Secretary                Health   
1                Administrative Officer    Financial Services   
2                    Recruiting Manager 

## Оцениваемые задания

### Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. (1 балл)

In [14]:
cursor.execute("""
    SELECT job_industry_category, COUNT(*) AS customer_count
    FROM Customer
    GROUP BY job_industry_category
    ORDER BY customer_count DESC;
""")

industry_distribution = cursor.fetchall()

for row in industry_distribution:
    print(row)


('Manufacturing', 799)
('Financial Services', 774)
('NaN', 656)
('Health', 602)
('Retail', 358)
('Property', 267)
('IT', 223)
('Entertainment', 136)
('Argiculture', 113)
('Telecommunications', 72)
(None, 1)


### Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. (1 балл)

In [15]:
cursor.execute("""
    SELECT 
        TO_CHAR(TO_DATE(t.transaction_date, 'DD.MM.YYYY'), 'YYYY-MM') AS transaction_month,
        c.job_industry_category,
        SUM(t.list_price) AS total_sales
    FROM Transaction t
    JOIN Customer c ON t.customer_id = c.customer_id
    GROUP BY transaction_month, c.job_industry_category
    ORDER BY transaction_month, c.job_industry_category;
""")

monthly_sales_by_industry = cursor.fetchall()

for row in monthly_sales_by_industry:
    print(row)


('2017-01', 'Argiculture', 43513.81999999999)
('2017-01', 'Entertainment', 64089.919999999984)
('2017-01', 'Financial Services', 366383.71000000054)
('2017-01', 'Health', 286860.3800000003)
('2017-01', 'IT', 107783.37)
('2017-01', 'Manufacturing', 365232.4500000004)
('2017-01', 'NaN', 316819.75000000023)
('2017-01', 'Property', 100686.97000000002)
('2017-01', 'Retail', 182375.7499999999)
('2017-01', 'Telecommunications', 31210.20000000001)
('2017-01', None, 1102.94)
('2017-02', 'Argiculture', 60016.80999999998)
('2017-02', 'Entertainment', 63965.98)
('2017-02', 'Financial Services', 375961.86000000004)
('2017-02', 'Health', 268525.7900000001)
('2017-02', 'IT', 93961.77000000002)
('2017-02', 'Manufacturing', 389260.5100000006)
('2017-02', 'NaN', 260711.90000000023)
('2017-02', 'Property', 112255.38999999998)
('2017-02', 'Retail', 146107.61999999997)
('2017-02', 'Telecommunications', 27678.2)
('2017-03', 'Argiculture', 49048.48999999997)
('2017-03', 'Entertainment', 77122.42999999998)
('

### Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. (1 балл)

In [16]:
# Найти количество онлайн-заказов для всех брендов среди подтвержденных заказов клиентов из сферы IT
cursor.execute("""
    SELECT 
        t.brand,
        COUNT(*) AS online_orders_count
    FROM Transaction t
    JOIN Customer c ON t.customer_id = c.customer_id
    WHERE t.online_order = 'true'
      AND t.order_status = 'Approved'
      AND c.job_industry_category = 'IT'
    GROUP BY t.brand
    ORDER BY online_orders_count DESC;
""")

# Получаем результат
online_orders_by_brand = cursor.fetchall()

# Выводим результат
for row in online_orders_by_brand:
    print(row)

('Solex', 101)
('Norco Bicycles', 92)
('WeareA2B', 90)
('Giant Bicycles', 89)
('Trek Bicycles', 82)
('OHM Cycles', 78)
('NaN', 8)


### Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. (2 балла)

In [17]:
cursor.execute("""
    SELECT 
        t.customer_id,
        SUM(t.list_price) AS total_spent,
        MAX(t.list_price) AS max_transaction,
        MIN(t.list_price) AS min_transaction,
        COUNT(t.transaction_id) AS total_transactions
    FROM Transaction t
    GROUP BY t.customer_id
    ORDER BY total_spent DESC, total_transactions DESC;
""")

group_by_result = cursor.fetchall()

for row in group_by_result:
    print(row)


(2183, 19071.32, 2005.66, 230.91, 14)
(1129, 18349.27, 1992.93, 290.62, 13)
(1597, 18052.68, 2091.47, 360.4, 12)
(941, 17898.46, 2091.47, 1057.51, 10)
(2788, 17258.94, 2083.94, 183.86, 11)
(936, 17160.24, 2005.66, 183.86, 12)
(1887, 17133.93, 2091.47, 688.63, 11)
(1302, 17035.829999999998, 1977.36, 71.16, 13)
(1140, 16199.240000000002, 2083.94, 183.86, 13)
(2309, 16122.339999999997, 2091.47, 290.62, 12)
(729, 15826.0, 2091.47, 586.45, 10)
(1103, 15447.920000000002, 1977.36, 230.91, 12)
(1317, 15370.809999999998, 2091.47, 569.56, 9)
(2874, 15091.910000000002, 2005.66, 544.05, 11)
(2762, 15071.26, 1997.68, 183.86, 10)
(255, 14949.91, 1810.0, 441.49, 12)
(1672, 14908.460000000001, 1977.36, 363.01, 13)
(1329, 14818.119999999999, 1793.43, 543.39, 11)
(322, 14782.23, 2005.66, 12.01, 12)
(3048, 14756.09, 1890.39, 543.39, 13)
(2860, 14673.35, 1812.75, 175.89, 10)
(999, 14662.139999999998, 2091.47, 495.72, 10)
(1460, 14633.24, 2091.47, 1228.07, 8)
(2894, 14621.73, 2083.94, 1129.13, 9)
(3060, 14

In [18]:
cursor.execute("""
    SELECT DISTINCT t.customer_id,
        SUM(t.list_price) OVER (PARTITION BY t.customer_id) AS total_spent,
        MAX(t.list_price) OVER (PARTITION BY t.customer_id) AS max_transaction,
        MIN(t.list_price) OVER (PARTITION BY t.customer_id) AS min_transaction,
        COUNT(t.transaction_id) OVER (PARTITION BY t.customer_id) AS total_transactions
    FROM Transaction t
    ORDER BY total_spent DESC, total_transactions DESC;
""")

window_function_result = cursor.fetchall()

for row in window_function_result:
    print(row)


(2183, 19071.32, 2005.66, 230.91, 14)
(1129, 18349.27, 1992.93, 290.62, 13)
(1597, 18052.679999999997, 2091.47, 360.4, 12)
(941, 17898.46, 2091.47, 1057.51, 10)
(2788, 17258.940000000002, 2083.94, 183.86, 11)
(936, 17160.239999999998, 2005.66, 183.86, 12)
(1887, 17133.93, 2091.47, 688.63, 11)
(1302, 17035.829999999998, 1977.36, 71.16, 13)
(1140, 16199.240000000002, 2083.94, 183.86, 13)
(2309, 16122.34, 2091.47, 290.62, 12)
(729, 15826.0, 2091.47, 586.45, 10)
(1103, 15447.92, 1977.36, 230.91, 12)
(1317, 15370.81, 2091.47, 569.56, 9)
(2874, 15091.91, 2005.66, 544.05, 11)
(2762, 15071.259999999998, 1997.68, 183.86, 10)
(255, 14949.91, 1810.0, 441.49, 12)
(1672, 14908.460000000001, 1977.36, 363.01, 13)
(1329, 14818.119999999999, 1793.43, 543.39, 11)
(322, 14782.23, 2005.66, 12.01, 12)
(3048, 14756.09, 1890.39, 543.39, 13)
(2860, 14673.350000000002, 1812.75, 175.89, 10)
(999, 14662.14, 2091.47, 495.72, 10)
(1460, 14633.239999999998, 2091.47, 1228.07, 8)
(2894, 14621.730000000001, 2083.94, 1

### Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы. (2 балла)

In [19]:
cursor.execute("""
    WITH total_spent AS (
        SELECT 
            c.customer_id,
            c.first_name,
            c.last_name,
            SUM(t.list_price) AS total_transactions
        FROM Customer c
        JOIN Transaction t ON c.customer_id = t.customer_id
        GROUP BY c.customer_id, c.first_name, c.last_name
    )
    SELECT first_name, last_name, total_transactions
    FROM total_spent
    WHERE total_transactions = (SELECT MAX(total_transactions) FROM total_spent);
""")

max_spending_customers = cursor.fetchall()

for row in max_spending_customers:
    print(row)


('Jillie', 'Fyndon', 19071.32)


In [20]:
cursor.execute("""
    WITH total_spent AS (
        SELECT 
            c.customer_id,
            c.first_name,
            c.last_name,
            SUM(t.list_price) AS total_transactions
        FROM Customer c
        JOIN Transaction t ON c.customer_id = t.customer_id
        GROUP BY c.customer_id, c.first_name, c.last_name
        HAVING SUM(t.list_price) IS NOT NULL  -- Исключаем NULL значения
    )
    SELECT first_name, last_name, total_transactions
    FROM total_spent
    WHERE total_transactions = (SELECT MIN(total_transactions) FROM total_spent);
""")

min_spending_customers = cursor.fetchall()

for row in min_spending_customers:
    print(row)


('Hamlen', 'Slograve', 60.34)


### Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. (1 балл)

In [21]:
cursor.execute("""
    SELECT customer_id, transaction_id, transaction_date, list_price, brand
    FROM (
        SELECT 
            t.customer_id,
            t.transaction_id,
            t.transaction_date,
            t.list_price,
            t.brand,
            ROW_NUMBER() OVER (PARTITION BY t.customer_id ORDER BY TO_DATE(t.transaction_date, 'DD.MM.YYYY')) AS row_num
        FROM Transaction t
    ) ranked_transactions
    WHERE row_num = 1;
""")

first_transactions = cursor.fetchall()

for row in first_transactions:
    print(row)


(1, 9785, '05.01.2017', 360.4, 'Norco Bicycles')
(2, 2261, '04.05.2017', 1403.5, 'Giant Bicycles')
(3, 10302, '23.02.2017', 1311.44, 'Giant Bicycles')
(4, 12441, '03.04.2017', 569.56, 'Giant Bicycles')
(5, 2291, '03.03.2017', 688.63, 'Norco Bicycles')
(6, 7096, '28.01.2017', 1769.64, 'WeareA2B')
(7, 18369, '18.02.2017', 360.4, 'Norco Bicycles')
(8, 10792, '04.01.2017', 1362.99, 'WeareA2B')
(9, 8591, '04.02.2017', 792.9, 'Giant Bicycles')
(10, 5956, '20.06.2017', 1071.23, 'Solex')
(11, 6004, '02.04.2017', 1216.14, 'Norco Bicycles')
(12, 6741, '12.02.2017', 360.4, 'Norco Bicycles')
(13, 17763, '06.01.2017', 1890.39, 'WeareA2B')
(14, 16253, '11.05.2017', 1636.9, 'OHM Cycles')
(15, 17528, '20.01.2017', 71.49, 'Solex')
(16, 14760, '12.02.2017', 1172.78, 'WeareA2B')
(17, 1404, '14.03.2017', 478.16, 'Solex')
(18, 7092, '25.03.2017', 202.62, 'Solex')
(19, 4664, '31.01.2017', 1311.44, 'Giant Bicycles')
(20, 18030, '11.06.2017', 2091.47, 'Trek Bicycles')
(21, 14115, '12.02.2017', 1538.99, 'Giant

### Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)  (2 балла)

In [22]:
cursor.execute("""
    WITH transaction_gaps AS (
        SELECT 
            c.customer_id,
            c.first_name,
            c.last_name,
            c.job_title,
            t.transaction_id,
            t.transaction_date,
            LAG(TO_DATE(t.transaction_date, 'DD.MM.YYYY')) OVER (
                PARTITION BY c.customer_id ORDER BY TO_DATE(t.transaction_date, 'DD.MM.YYYY')
            ) AS prev_transaction_date
        FROM Transaction t
        JOIN Customer c ON t.customer_id = c.customer_id
    )
    SELECT first_name, last_name, job_title, 
           MAX(TO_DATE(transaction_date, 'DD.MM.YYYY') - prev_transaction_date) AS max_interval
    FROM transaction_gaps
    WHERE prev_transaction_date IS NOT NULL
    GROUP BY first_name, last_name, job_title
    ORDER BY max_interval DESC
""")

# Получаем результат
max_gap_customer = cursor.fetchall()

# Выводим результат
for row in max_gap_customer:
    print(row)


('Susanetta', 'NaN', 'Legal Assistant', 357)
('Stoddard', 'Giacomoni', 'Structural Analysis Engineer', 330)
('Gregorius', 'Cockram', 'Data Coordiator', 330)
('Royall', 'Terris', 'Geological Engineer', 330)
('Bearnard', 'Letixier', 'NaN', 329)
('Caralie', 'Sellors', 'Senior Editor', 321)
('Debee', 'Martynov', 'Senior Editor', 320)
('Genni', 'Larway', 'Environmental Specialist', 314)
('Carolynn', 'Samsin', 'Pharmacist', 310)
('Franz', 'Craddy', 'NaN', 310)
('Timmie', 'Lenden', 'NaN', 310)
('Ashia', 'Muzzi', 'Mechanical Systems Engineer', 306)
('Jodee', 'Judkins', 'Recruiting Manager', 306)
('Heywood', 'Sollett', 'Tax Accountant', 305)
('Cleveland', 'Islep', 'Software Engineer II', 299)
('Sheilah', 'Blackmore', 'NaN', 297)
('Michel', "O'Halligan", 'Software Test Engineer IV', 297)
('Jazmin', 'Neumann', 'VP Quality Control', 297)
('Jesus', 'MacShirie', 'NaN', 296)
('Donny', 'Stiven', 'Computer Systems Analyst II', 292)
('Emmery', 'Angrock', 'Information Systems Manager', 292)
('Osborne', '